<a href="https://colab.research.google.com/github/sasimsb/ML-Challenge-3/blob/master/Summary_of_the_Data_and_Upload_to_Google_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the libraries and installing packages if needed**

In [9]:
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
except:
  "pass"

Mounted at /content/gdrive


In [11]:
import pandas as pd
import numpy as np
import os
import warnings 
warnings.filterwarnings('ignore')
import time
import datetime
!pip install xlsxwriter
import xlsxwriter
import openpyxl

     |████████████████████████████████| 153kB 6.2MB/s 


**Loading the excel file here or we can change the path of file**

In [12]:
## change your path location based on your path file present
working_dir = '/content/gdrive/MyDrive/project2/'
os.chdir(working_dir)
## change the file name based on what is present
data = pd.read_excel('Analysis_v03.xlsx', sheet_name="Original_Data")

**Any Unncessary columns to delete**

In [13]:
try:
  del data['Unnamed: 18']
  del data['Unnamed: 19']
  del data['Unnamed: 20']
  del data['Unnamed: 21']
  del data['Unnamed: 22']
  del data['Unnamed: 23']
  del data['Unnamed: 24']
  del data['Unnamed: 25']
  del data['Unnamed: 26']
  del data['Unnamed: 27']
  del data['Unnamed: 28']
  del data['Unnamed: 29']
  del data['Unnamed: 30']
  del data['County Selection']
  del data['State Selection']
  del data['Resi/Com Selection']
  del data['Division.1']  
except:
  pass

**Check on the columns that are to be numeric but they are categorical** 

In [14]:
try:
  data['Instrument Number'] = pd.to_numeric(data['Instrument Number'])
except:
  print("some characters are present so it wont convert to numeric")

some characters are present so it wont convert to numeric


In [15]:
try:
  data['Total Value'] = pd.to_numeric(data['Total Value'])
except:
  print("some characters are present so it wont convert to numeric")

some characters are present so it wont convert to numeric


In [16]:
try:
  data['Lien Amount'] = pd.to_numeric(data['Lien Amount'])
except:
  print("some characters are present so it wont convert to numeric")

some characters are present so it wont convert to numeric


**Column changing from categorical to numeric done here**

In [17]:
data3 = data[data['Instrument Number'].notna()]

**Steps to convert the data from categorical to numeric**
1. convert the column to string format
2. replace the data from "." with "-"
3. try to add "-" at the end so that no issue with other rows 
4. convert the data again to string format (check)
5. split the data by extracting the information from the last "-" that is present
6. replace the null values with 0 
7. convert the column to numeric from categorical

In [18]:
data3['Total Value'] = data3['Total Value'].astype(str)
data3['Total Value1'] =data3['Total Value'].str.replace(".","-")
data3['Total Value2'] = data3['Total Value1']+"-"
data3['Total Value2']= data3['Total Value2'].astype(str)
data3['Total Value3'] = data3['Total Value2'].apply(lambda x: x.split('-')[0])
data3['Total Value3'] = data3['Total Value3'].replace("nan",0)
data3['Total Value3'] =  pd.to_numeric(data3['Total Value3'])
del data3['Total Value1']
del data3['Total Value2']

In [19]:
data3['Lien Amount'] = data3['Lien Amount'].astype(str)
data3['Lien Amount'] =data3['Lien Amount'].str.replace("$","")
data3['Lien Amount'] =data3['Lien Amount'].str.replace(",","")
data3['Lien Amount'] =data3['Lien Amount'].str.replace(" ","")
data3['Lien Amount1'] =data3['Lien Amount'].str.replace(".","-")
data3['Lien Amount2'] = data3['Lien Amount1']+"-"
data3['Lien Amount2']= data3['Lien Amount2'].astype(str)
data3['Lien Amount3'] = data3['Lien Amount2'].apply(lambda x: x.split('-')[0])
data3['Lien Amount3'] = data3['Lien Amount3'].replace("nan",0)
data3['Lien Amount3'] =  pd.to_numeric(data3['Lien Amount3'])
del data3['Lien Amount1']
del data3['Lien Amount2']

***New variables creations ***
1. Lien amount percentage 
2. lien amount bins

In [20]:
data3['lien_amount_Percent'] = data3['Lien Amount3']/data3['Total Value3']*100

In [21]:
conditions = [
    data3['lien_amount_Percent'] >= 100,
    ((data3['lien_amount_Percent'] > 90.00) & (data3['lien_amount_Percent'] <= 99.00)),
    ((data3['lien_amount_Percent'] > 80.00) & (data3['lien_amount_Percent'] <=90.00)),
    ((data3['lien_amount_Percent'] > 70.00) & (data3['lien_amount_Percent'] <= 80.00)),
    ((data3['lien_amount_Percent'] > 60.00) & (data3['lien_amount_Percent'] <=70.00)),
    ((data3['lien_amount_Percent'] > 50.00) & (data3['lien_amount_Percent'] <= 60.00)),
    ((data3['lien_amount_Percent'] > 40.00) & (data3['lien_amount_Percent'] <=50.00)),
    ((data3['lien_amount_Percent'] > 30.00) & (data3['lien_amount_Percent'] <= 40.00)),
    ((data3['lien_amount_Percent'] > 20.00) & (data3['lien_amount_Percent'] <=30.00)),
    ((data3['lien_amount_Percent'] > 10.00) & (data3['lien_amount_Percent'] <=20.00))
]
choices =['100%',"90%-100%", '80-90','70-80','60-70','50-60','40-50','30-40','20-30','10-20']
data3['lien_amount_bins'] = np.select(conditions,choices, 'less than 10%')
data3['Lien Amount 100%'] = np.where(data3['lien_amount_bins']=='100%', "Yes", "No")

***Creating the date variables ***
1. Day of the week it is recorded
2. Month of the date it is recorded

In [22]:
data3['Date Recorded'] = data3['Date Recorded'].astype(str)
data3['Date Recorded'] = data3['Date Recorded'].replace('3021-03-04 00:00:00','2021-03-04 00:00:00')
data3['Date Recorded'] = pd.to_datetime(data3['Date Recorded'])
data3['Date'] = [d.date() for d in data3['Date Recorded']]
data3['Date'] = pd.to_datetime(data3['Date'], errors='coerce')
data3["Recorded_Month"] = data3["Date"].dt.month_name()
data3['Recorded_Days'] = data3['Date'].dt.day_name()
del data3['Date']

**Creating a new variable decile**

In [23]:
data3['Decile'] = pd.qcut(data3['Total Value3'], 10,labels = False)
data3['Decile']= data3['Decile']+1

In [24]:
#data4 = data3.drop(data3.columns[[13,14]], axis = 1)
data4 = data3
del data4['Total Value']
del data4['Lien Amount']

In [25]:
data4.head(2)

,Date Recorded,County,Instrument Number,Lienor,Contract With,Project Address,City,State,Zip,Owner,Resi/Com,Division,Total Value3,Lien Amount3,lien_amount_Percent,lien_amount_bins,Lien Amount 100%,Recorded_Month,Recorded_Days,Decile
0,2021-01-11,Miami-Dade,20210018890,"Florida Block Carpentry , Inc.",Steve Bass,1120 Wallace Street Coral Gables Florida,Coral Gables,FL,NaN,Steve Bass & Vanessa Bass,Residential,General Contracting,2240,2240,100.000000,100%,Yes,January,Monday,1
1,2021-01-11,Miami-Dade,20210020825,"Blckstone Consulting , LLC.",Atlantic Coast Restaurant & Mechanical Service...,2820 NE 214 ST.,Aventura,FL,33180.0,Shaley Development Group LLC.,Commercial,Architectural/Engineering/Surveying,67352,31139,46.233222,40-50,No,January,Monday,8


In [26]:
data4['Zip'] = data4['Zip'].astype(str)
data4['Date Recorded'] = data4['Date Recorded'].astype(str)

**Calculation of summary and missing value percentage**

In [27]:
x = data4.describe(include="all").T.reset_index()
y = (data4.isnull().sum()/len(data4)*100).reset_index(name="Missing %")
final_data = pd.merge(x,y, on='index', how='left')
final_data = final_data.rename(columns={'index':'column_name'})

In [28]:
final_data.head(3)

,column_name,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max,Missing %
0,Date Recorded,1893,86,2021-02-02 00:00:00,40,2021-01-04,2021-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,County,1893,3,Miami-Dade,720,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Instrument Number,1893,1882,2.02101e+10,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


**Putting the columns in order and renaming if needed**

In [29]:
final_data1 = final_data[["column_name",'count', 'unique','Missing %','top','freq','mean','std','min','25%','50%','75%','max']]

In [30]:
data4 = data4[['Date Recorded','Recorded_Month', 'Recorded_Days', 'County', 'Instrument Number', 'Lienor',
       'Contract With', 'Project Address', 'City', 'State', 'Zip', 'Owner',
       'Resi/Com', 'Division', 'Total Value3', 'Lien Amount3',
       'lien_amount_Percent', 'lien_amount_bins', 'Lien Amount 100%',
        'Decile']]

In [31]:
data4 = data4.rename(columns= {'Total Value3':'Total Value',
                               'Lien Amount3':'Lien Amount',
                               'lien_amount_Percent':'%Lien Amount',
                               'lien_amount_bins':'%Lien Amount Bin'})

**writing the final data to excel**

In [40]:
writer = pd.ExcelWriter('/content/gdrive/MyDrive/project2/final_data_v2.xlsx', engine='xlsxwriter')
data.to_excel(writer, sheet_name='Original_Data',index=False)
data4.to_excel(writer, sheet_name='Refined_Data', index=False)
final_data1.to_excel(writer, sheet_name="Summary of Data",index=False)
writer.save()
writer.close()

**End of the code**

**Method to upload data to google sheets and then get the plots directly**

In [72]:
import gspread
import pandas as pd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

def iter_pd(df):
    for val in df.columns:
        yield val
    for row in df.to_numpy():
        for val in row:
            if pd.isna(val):
                yield ""
            else:
                yield val

def pandas_to_sheets(pandas_df, sheet, clear = True):
    # Updates all values in a workbook to match a pandas dataframe
    if clear:
        sheet.clear()
    (row, col) = pandas_df.shape
    cells = sheet.range("A1:{}".format(gspread.utils.rowcol_to_a1(row + 1, col)))
    for cell, val in zip(cells, iter_pd(pandas_df)):
        cell.value = val
    sheet.update_cells(cells)

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/gdrive/MyDrive/project2/jsonFileFromGoogle.json', scope)


In [74]:
gc = gspread.authorize(credentials)
workbook = gc.open_by_key("1E3yqB3oeNlcGVubKknqXDfU2uDKIm3zdrWiY5hB6rpU")
pandas_to_sheets(data4, workbook.worksheet("Refined_Data"))

In [82]:
final_data2= final_data1.fillna(0)

In [91]:
final_data1['Missing %'] = final_data1['Missing %'].astype(float)
final_data1['count'] = final_data1['count'].astype(float)
final_data1['unique'] = final_data1['unique'].astype(float)
final_data1['freq'] = final_data1['freq'].astype(float)
final_data1['mean'] = final_data1['mean'].astype(float)
final_data1['std'] = final_data1['std'].astype(float)
final_data1['min'] = final_data1['min'].astype(float)
final_data1['25%'] = final_data1['25%'].astype(float)
final_data1['50%'] = final_data1['50%'].astype(float)
final_data1['75%'] = final_data1['75%'].astype(float)
final_data1['max'] = final_data1['max'].astype(float)
final_data1['top'] = final_data1['top'].astype(str)

In [92]:
pandas_to_sheets(final_data1, workbook.worksheet("Summary"))

End of Updating the Sheet